### Constraints
1. num of pianists, 2 hands, 5 fingers
2. time apart
3. spatial apart (in each hand cannot exceed one octave)
4. within one octave > same intensity > notes start at the same time > last for same period > not adjacent

In [ ]:
import numpy as np
import scipy.io
import matplotlib.pyplot as plt
from midiutil import MIDIFile
from pprint import pprint

from utils import *

import copy
import random

In [ ]:
## Parameters
group_num = 21 * 2 #21 pianists, 2 hands each
max_note_per_group = 5 #5 fingers

#min_pitch_gap = not set
win_length = 50e-3
max_time_gap = 0.3
max_frame_gap = int(round(max_time_gap / win_length))

In [ ]:
key_time = 50e-3*0.5
Fs = 48000

## picked keys
keys = scipy.io.loadmat('MATLAB_data/23Feb_jc_key_2octave.mat')['key']

##freq of notes on piano
notes_list = scipy.io.loadmat('notes_112.mat')['notes'] 

##freq of t-f components
freq_list = scipy.io.loadmat('freq_480_two_octave_down.mat')['f_cropped'] 

intensity_map = scipy.io.loadmat('MATLAB_data/jw_int.mat')['filtered_int_db']
notes_list = notes_list.reshape(np.size(notes_list))
freq_list = freq_list.reshape(np.size(freq_list))

note_num = 112

notes = []
for i in range(np.shape(keys)[0]):
    cur_key = keys[i]
    cur_int = intensity_map[i]
    cur_freq = freq_list[i]
    notes = picknotes(cur_key, cur_freq, cur_int, notes, notes_list)

notes = sorted(notes, key = lambda note: note.start)

new_notes = remove_repetitive(notes,note_num)
#pprint(new_notes)
separate_majors(new_notes)


In [ ]:
pprint(new_notes)

In [ ]:
word_gap = 15
def separate_word(notes,word_gap):
    word_list = []
    ptr = 0
    for i in range(len(notes)-1):
        if notes[i+1].start - notes[i].end >= word_gap:
            word_list.append(notes[ptr:i+1])
            ptr = i + 1
        if i == len(notes) - 2:
            word_list.append(notes[ptr:])
    return word_list
    

In [ ]:
def valid_note_to_allocate(note,overlapped,pitch_group,high):
    low = note.pitch if pitch_group == [] else min(pitch_group)
    
    if (note.pitch not in pitch_group) and (note.pitch-low <= 12):
        #q remain: limit highest note range?
        if overlapped and note.pitch < (high + 1):
            return False
        return True
    return False

def valid_to_add(counter,candidate,current_group):
    c1 = counter < len(candidate)
    c2 = len(current_group) < 5
    return c1 and c2

In [ ]:
def priority(note, current_allocated_note,major):
    p = 0
    # if note.major == major:
    #     p += 3

    if note.intensity_sign == current_allocated_note.intensity_sign:
        p += 4
    
    if note.start == current_allocated_note.start and note.end == current_allocated_note.end:
        p += 2
    
    return p

In [ ]:
def check_handshape(allocated_group,overlapped):
    n = len(allocated_group) #number of notes in the group
    pitchs = [note.pitch for note in allocated_group]
    pitchs.sort()
    constraints = {'12': 9, '13': 12, '14': 12, '15': 12, '23': 7, '24': 8, '25': 10, '34': 5, '35': 7, '45': 4}
    if n < 3:
        return True
    if overlapped:
        #right hand
        if n == 3:
            if pitchs[-1]-pitchs[-2] > 9: return False
            return True
        if n == 4:
            g1 = pitchs[1] - pitchs[0] 
            g2 = pitchs[2] - pitchs[1] 
            g3 = pitchs[3] - pitchs[2] 
            g4 = pitchs[2] - pitchs[0]
            g5 = pitchs[3] - pitchs[1]
            g6 = pitchs[3] - pitchs[0]
            #1234
            if g1 <= constraints['12'] and g2 <= constraints['23'] and g3 <= constraints['34'] and g4 <= constraints['13'] and g5 <= constraints['24'] and g6 <= constraints['14']:
                return True
            #1235
            elif g1 <= constraints['12'] and g2 <= constraints['23'] and g3 <= constraints['35'] and g4 <= constraints['13'] and g5 <= constraints['25'] and g6 <= constraints['15']:
                return True
            #1245
            elif g1 <= constraints['12'] and g2 <= constraints['24'] and g3 <= constraints['45'] and g4 <= constraints['14'] and g5 <= constraints['25'] and g6 <= constraints['15']:
                return True
            #1345
            elif g1 <= constraints['13'] and g2 <= constraints['34'] and g3 <= constraints['45'] and g4 <= constraints['14'] and g5 <= constraints['35'] and g6 <= constraints['15']:
                return True
            #2345
            elif g1 <= constraints['23'] and g2 <= constraints['34'] and g3 <= constraints['45'] and g4 <= constraints['24'] and g5 <= constraints['35'] and g6 <= constraints['25']:
                return True
            return False
        if n == 5:
            g1 = (pitchs[1] - pitchs[0]) <= constraints['12']
            g2 = (pitchs[2] - pitchs[0]) <= constraints['13']
            g3 = (pitchs[3] - pitchs[0]) <= constraints['14']
            g4 = (pitchs[4] - pitchs[0]) <= constraints['15']
            g5 = (pitchs[2] - pitchs[1]) <= constraints['23']
            g6 = (pitchs[3] - pitchs[1]) <= constraints['24']
            g7 = (pitchs[4] - pitchs[1]) <= constraints['25']
            g8 = (pitchs[3] - pitchs[2]) <= constraints['34']
            g9 = (pitchs[4] - pitchs[2]) <= constraints['35']
            g10 = (pitchs[4] - pitchs[3]) <= constraints['45']
            return all([g1, g2, g3, g4, g5, g6, g7, g8, g9, g10])
    else:
        #left hand
        if n == 3:
            if pitchs[1]-pitchs[0] > 9: return False
            return True
        if n == 4:
            g1 = pitchs[1] - pitchs[0] 
            g2 = pitchs[2] - pitchs[1] 
            g3 = pitchs[3] - pitchs[2] 
            g4 = pitchs[2] - pitchs[0]
            g5 = pitchs[3] - pitchs[1]
            g6 = pitchs[3] - pitchs[0]
            #4321
            if g1 <= constraints['34'] and g2 <= constraints['23'] and g3 <= constraints['12'] and g4 <= constraints['24'] and g5 <= constraints['13'] and g6 <= constraints['14']:
                return True
            #5321
            elif g1 <= constraints['35'] and g2 <= constraints['23'] and g3 <= constraints['12'] and g4 <= constraints['25'] and g5 <= constraints['13'] and g6 <= constraints['15']:
                return True
            #5421
            elif g1 <= constraints['45'] and g2 <= constraints['24'] and g3 <= constraints['12'] and g4 <= constraints['25'] and g5 <= constraints['14'] and g6 <= constraints['15']:
                return True
            #5431
            elif g1 <= constraints['45'] and g2 <= constraints['34'] and g3 <= constraints['13'] and g4 <= constraints['35'] and g5 <= constraints['14'] and g6 <= constraints['15']:
                return True
            #5432
            elif g1 <= constraints['45'] and g2 <= constraints['34'] and g3 <= constraints['23'] and g4 <= constraints['35'] and g5 <= constraints['24'] and g6 <= constraints['25']:
                return True
            return False
        if n == 5:
            g1 = (pitchs[1] - pitchs[0]) <= constraints['45']
            g2 = (pitchs[2] - pitchs[0]) <= constraints['35']
            g3 = (pitchs[3] - pitchs[0]) <= constraints['25']
            g4 = (pitchs[4] - pitchs[0]) <= constraints['15']
            g5 = (pitchs[2] - pitchs[1]) <= constraints['34']
            g6 = (pitchs[3] - pitchs[1]) <= constraints['24']
            g7 = (pitchs[4] - pitchs[1]) <= constraints['14']
            g8 = (pitchs[3] - pitchs[2]) <= constraints['23']
            g9 = (pitchs[4] - pitchs[2]) <= constraints['13']
            g10 = (pitchs[4] - pitchs[3]) <= constraints['12']
            return all([g1, g2, g3, g4, g5, g6, g7, g8, g9, g10])
    print('Group not valid')
    return False

In [ ]:
note1 = Note(40,0,0,0)
note2 = Note(44,0,0,0)
note3 = Note(47,0,0,0)
note4 = Note(50,0,0,0)
note5 = Note(52,0,0,0)
testgroup = [note1,note2,note3,note4,note5]
print(check_handshape(testgroup,True))

In [ ]:
def allocation(notes_group):
    allocate_result=[]
    group = 1
    overlapped = False #true if last group was left hand, current group is right hand, they should not overlap
    while notes_group != [] and group <= group_num:
        ## each while iteration allocate one group
        current_group = []
        pitch_group = []
        
        if not overlapped:
            high = 0
            minnote = min(notes_group, key = lambda note: note.pitch)
            pitch_group = [minnote.pitch]
            major = minnote.major
            current_group.append(minnote)
            notes_group.remove(minnote)
        else:
            for i in range(len(notes_group)):
                if notes_group[i].pitch > (high + 1):
                    pitch_group = [notes_group[i].pitch]
                    current_group.append(notes_group.pop(i))
                    break
        candidate = []            
        for current_allocated_note in current_group:
            candidate = []
            # counter = 0
            for note in notes_group:
                if not valid_note_to_allocate(note,overlapped,pitch_group,high):
                    continue
                p = priority(note,current_allocated_note,major)
                candidate.append([p,note])
            candidate.sort(key = lambda x: x[0], reverse=True)
            #print(candidate)
            counter = 0
            while valid_to_add(counter,candidate,current_group):
                if  (candidate[counter][1].pitch not in pitch_group) and (candidate[counter][0] > 4):
                    pitch_group.append(candidate[counter][1].pitch)
                    current_group.append(candidate[counter][1])
                    notes_group.remove(candidate[counter][1])
                counter += 1

        counter = 0
        while valid_to_add(counter,candidate,current_group):
            if (candidate[counter][1].pitch not in pitch_group):
                pitch_group.append(candidate[counter][1].pitch)
                current_group.append(candidate[counter][1])
                if candidate[counter][1] not in notes_group:
                    print(candidate[counter][1])
                notes_group.remove(candidate[counter][1])
            counter += 1

        if not overlapped:
            high = max([n.pitch for n in current_group])

        allocate_result.append(current_group)

        overlapped = not overlapped
        group += 1 #finish allocate notes to one group, update
    
    counter = 0
    while counter < len(notes_group):
        note = notes_group[counter]
        for i, allocated_group in enumerate(allocate_result):
            if len(allocated_group) >= 4:
                continue
            if len(allocated_group) < 4:
                pitch_group = [n.pitch for n in allocated_group]
                high = max([n.pitch for n in allocate_result[i-1]]) if i % 2 == 0 else 0
                overlapped = True if i % 2 == 0 else False
                if valid_note_to_allocate(note,overlapped,pitch_group,high):
                    allocated_group.append(note)
                    notes_group.remove(note)
                    break
        counter += 1

    if notes_group != []:
        print('Failed :(    ',len(notes_group), 'notes unallocated.')
        return False
    else:
        print('Success!')
        return allocate_result

In [ ]:
word_groups = separate_word(new_notes,word_gap)
for item in word_groups:
    print(len(item))

In [ ]:
# word_groups = separate_word(new_notes)
final_allocation1 = [[],[]]
final_allocation2 = [[],[]]
result_all = []
for i in range(2,5):
    print(i)
    current_allocate_group = copy.deepcopy(word_groups[i])
    #random.shuffle(current_allocate_group)
    result = False
    while not result:
        result = allocation(current_allocate_group)
        current_allocate_group = copy.deepcopy(word_groups[i])
        random.shuffle(current_allocate_group)
    result_all.append(result)
    final_allocation1[0].extend(result[0])
    final_allocation1[1].extend(result[1])
    final_allocation2[0].extend(result[20])
    final_allocation2[1].extend(result[21])
    pprint(final_allocation1)
    pprint(final_allocation2)

    

In [ ]:
current_allocate_group = copy.deepcopy(word_groups[4])
result = False
while not result:
    result = allocation(current_allocate_group)
    current_allocate_group = copy.deepcopy(word_groups[4])
    random.shuffle(current_allocate_group)


In [ ]:
plot_notes(new_notes)
plt.show()
word_groups = separate_word(new_notes)
for g in word_groups:
    plot_notes(g)
plt.show()

In [ ]:
mf = MIDIFile(2)     # track number
time = 0    # start at the beginning
mf.addTrackName(0, time, "Right")
mf.addTrackName(1, time, "Left")
mf.addTempo(0, time, 1200)
mf.addTempo(1, time, 1200)

In [ ]:
for i in range(2):    
    for note in final_allocation2[1-i]:
        # v = int(50+note.sign()*12.5)
        v = int(20+note.sign()*20)
        mf.addNote(i,i,note.pitch+20, note.start, note.end-note.start+1,v)

In [ ]:
with open("midifile/23Feb_allocation2.mid", 'wb') as outf:
   mf.writeFile(outf)

In [ ]:
pprint(final_allocation)

In [ ]:
import numpy as np
array = np.array(result_all,dtype=object)
np.save("allocation_result.npy", array)